# Impureza de Gini

### La impureza de Gini es una magnitud de incertidumbre, esto se refiere a la aleatoriedad con la que se elige un elemento $i$ del conjunto de datos y la probabilidad de asignarle al elemento $i$ una etiqueta $m$ de forma incorrecta.

# $$Gini = 1 - \sum_{i}^{c} (p_{i})^{2}$$


## Análisis de la función.

### Para cada clase se deberá calcular la probabilidad que le corresponda, esta es la probabilidad que posee cada clase de ser escogida y se representa como $p_{i}$. Estas probabilidades de cada clase se pueden calcular por separado. 

### Por lo tanto, para calcular la impureza de Gini, crearemos primero (y por separado) una función que calcule las $p_{i}$'s, para así poder usarlas almacenando en un diccionario de Python los valores de las $k$ clases.

### Ahora bien, a partir de esta función que calcula las probabilidades, almacenamos en una nueva variable $P = \sum (p_{i})^2$ que es la suma las probabilidades al cuadrado. Si se aplica $1 - P$ esto representa el _complemento_ de la probabilidad $P$ i.e. la parte _contraria de la probabilidad_. A partir de tener estos cálculos ya podemos seguir a Calcular tanto Gini como la entropía.

### Al $1 - P$, que obtuvimos en el paso anterior, la consideramos una maginitud contraria a la suma de las probabilidades al cuadrado $P$. $1 - P$ describe la probabilidad de ser etiquetado de forma incorrecta por el nivel de impureza en los datos, de aquí nace la idea de la _magnitud de incertidumbre_, i.e. _Gini_.

<font size="4">**Por ahora crearemos nuestros nodos los cuales almacenarán información de cada clase**</font> 

In [3]:
Root_node = {"Manzana": 100, "Platano": 400, "Mango": 5, "Uva": 550}
child_node_st = {"Manzana": 100, "Platano": 400, "Mango": 5}
child_node_nd = {"Platano": 400, "Mango": 5}
child_node_rd = {"Platano": 400}

### Lo que se pretende mostrar con este ejercicio *dummy* es el nivel de información que aporta cada clase y el uso de las probabilidades de cada una. Pretendemos que sea un ejercicio mental básico, pero que soporta una ley que hay detrás de estas dos funciones: la ley de probabilidad.

In [12]:
# Ejercicio Dummy.
print("Datos de entrada:", Root_node)
print("Suma Total:", 100+400+5+550)
print("Prob Manzana:", 100/1055)
print("Prob Platano:", 400/1055)
print("Prob Mango:", 5/1055)
print("Prob Uva:", 550/1055)
print("\nSuma de probabilidades:", (100/1055)+(400/1055)+(5/1055)+(550/1055))

Datos de entrada: {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550}
Suma Total: 1055
Prob Manzana: 0.0947867298578199
Prob Platano: 0.3791469194312796
Prob Mango: 0.004739336492890996
Prob Uva: 0.5213270142180095

Suma de probabilidades: 1.0


## ¡Ok! Suficiente, programemos las funciones...

In [4]:
def probability_class(node):
    node_sum = sum(node.values())
    percents = {c:v / node_sum for c, v in node.items()}
    return node_sum, percents

### Gini requiere menos costo computacional que el cálculo de la Entropía, debido al menor número de operaciones aritméticas que realiza la función (la suma de las probabilidades al cuadrado). Sin embargo, a las *probabilidades menores casi no las toma en cuenta* y éstas tienden a influir muy poco (o casi nada) en el resultado final.

## _Cuenta demostrativa:_

### $$0.0948^{2} + 0.3791^{2} + 0.0047^{2} + 0.5213^{2}$$

### $$0.009 + 0.1438 + 0.00002209 + 0.2718$$

### $$P = 0.4246$$

### $$Gini = 1 - P = 0.5754$$

### La función `gini score` es la función por la que obtendremos la magnitud Gini (explicada al inicio del notebook), para ello en `gini_score` agregamosla función `probability_class` para mejorar y atomizar ambos comportamientos.

In [6]:
def gini_score(node):
    _, percents = probability_class(node)
    # donde i contiene la probabilidad calculada del nodo en cuestión
    score = round(1 - sum([i**2 for i in percents.values()]), 3)
    print('Gini Score for node {} : {}'.format(node, score))
    return score

# Entropía

## Análisis de la función

### La función de Entropía es menos eficiente en términos computacionales dado que involucra el uso de $\log_{2}$, pero lo interesante aquí es que como usa el $\log$ a las probabilidades menores y mayores, las ajusta y pondera las probabilidades que son pequeñas. Por ello, el resultado será mucho más sensible a estas probabilidades pequeñas que en el casro del score de Gini. En los casos que se busque darle relevancia a las probabilidades pequeñas, se recomienda usar la Entropía como medida de impureza en lugar del score de Gini.

# $$E(S) = \sum_{i}^{c} - p_{i} \log_{2} p_{i}$$

## _Cuenta demostrativa:_

### $$- 0.0948 \log_{2}(0.0948) + -0.3791 \log_{2}(0.3791) + -0.0047 \log_{2}(0.0047) + -0.5213 \log_{2}(0.5213)$$

### $$0.3222 + 0.5304 + 0.0363 + 0.4899$$

### $$Entropy = 1.3788$$

In [2]:
from math import log

In [7]:
def entropy_score(node):
    _, percents = probability_class(node)
    # donde i contiene la probabilidad calculada del nodo en cuestión
    score = round(sum([-i * log(i, 2) for i in percents.values()]), 3)
    print('Entropy Score for node {} : {}'.format(node, score))
    return score

### Ahora calcularemos la ganancia de información (`information_gain`), a menor impureza, mayor ganancia de información.

In [8]:
def information_gain(parent, children, criterion):
    score = {'gini': gini_score, 'entropy': entropy_score}
    metric = score[criterion]
    parent_score = metric(parent)
    parent_sum = sum(parent.values())
    weighted_child_score = sum([metric(i) * sum(i.values()) / parent_sum  for i in children])
    gain = round((parent_score - weighted_child_score),2)
    print('Information gain: {}'.format(gain))
    return gain

### **Aquí mostramos el funcionamiento tanto de Gini como de la Entropía. En este caso el número 5 quiere decir que estamos incluyendo solamente 5 registros que contienen la clase "Mango", por lo tanto podemos notar que hay un importante desbalance entre las cantidades por cada clase, por ejemplo:**

- En el caso de Mazana, Platano y Uva, éstas contienen cantidades por encima de los 100 registros, claramente son las clases predominantes en este ejemplo, por lo que serán de gran influencia en la separación de la información en cada nodo hijo

- En el caso de la clase Mango, podemos suponer que la probabilidad calculada para ella será muy pequeña, por lo que será muy probable que la función de magnitud Gini ignore dicha clase

- Por otro lado vemos que la función de Entropia, no solo logra distinguir a la clase minoritaria, sino que separa a mayor precisión la información que reside en cada clase dando como resultado final una mayor ganancia de información.

In [9]:
gini_gain = information_gain(parent=Root_node, children=[child_node_st, child_node_nd, child_node_rd], criterion='gini')

Gini Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550} : 0.575
Gini Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5} : 0.333
Gini Score for node {'Platano': 400, 'Mango': 5} : 0.024
Gini Score for node {'Platano': 400} : 0.0
Information gain: 0.41


In [10]:
entropy_gain = information_gain(parent=Root_node, children=[child_node_st, child_node_nd, child_node_rd], criterion='entropy')

Entropy Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5, 'Uva': 550} : 1.379
Entropy Score for node {'Manzana': 100, 'Platano': 400, 'Mango': 5} : 0.795
Entropy Score for node {'Platano': 400, 'Mango': 5} : 0.096
Entropy Score for node {'Platano': 400} : 0.0
Information gain: 0.96


## Referencias:
- Anónimo. (2020). Entropía, Gini. 2020, de Numerentur Sitio web: http://numerentur.org/entropia-indice-de-ganancia/
- Anónimo. (2020). Índice Gini. 2021, de Intercactive Chaos Sitio web: https://interactivechaos.com/es/manual/tutorial-de-machine-learning/indice-gini